In [1]:
%pip install pandas
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"device is {device}")

Note: you may need to restart the kernel to use updated packages.
device is cpu


In [2]:
#downloading the data

# import requests

# url = 'https://zenodo.org/records/4561253/files/WELFake_Dataset.csv'  
# response = requests.get(url)

# with open('news_data.csv', 'wb') as file:
#     file.write(response.content)

# print("downloading finished")

In [3]:
import pandas as pd
df = pd.read_csv('news_data.csv')
df.head()


,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [ ]:


df_sample = df.sample(n=4000, random_state=42) 

df_sample.to_csv('news_data_4000.csv', index=False)

In [4]:
df.info()
df.isnull().sum()
df.describe()
df.columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  124 non-null    int64 
 1   title       122 non-null    object
 2   text        124 non-null    object
 3   label       124 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 4.0+ KB


Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')

In [5]:
#removing duds

df = df.dropna()
df = df.drop(columns=['Unnamed: 0'])
df['label'] = pd.to_numeric(df['label'], errors='coerce')
df.head()
df = df.drop_duplicates()
df = df.dropna(subset=['text', 'title'])


In [6]:
#some regex cleaning

import re

def cleanup(text):
    if pd.isna(text):
        return ""
    text = text.strip()  
    #remove any web urls
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) 
    #remove email addresses
    text = re.sub(r'\b\w+@\w+\.\w+\b', '', text)  
    #get rid of any duplicate spacing
    text = re.sub(r'\s+', ' ', text)  
    #get rid of punctuaation
    text = re.sub(r'[^\w\s]', '', text) 
    #lowercase all text
    text = text.lower() 
    return text

df['text'] = df['text'].apply(cleanup)
df['title'] = df['title'].apply(cleanup)

In [7]:
df.head()

,title,text,label
0,law enforcement on high alert following threat...,no comment is expected from barack obama membe...,1
2,unbelievable obamas attorney general says most...,now most of the demonstrators gathered last ni...,1
3,bobby jindal raised hindu uses story of christ...,a dozen politically active pastors came here f...,0
4,satan 2 russia unvelis an image of its terrify...,the rs28 sarmat missile dubbed satan 2 will re...,1
5,about time christian group sues amazon and spl...,all we can say on this one is it s about time ...,1


In [8]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [9]:
#removing any stop words

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stop_words(text):
    return ' '.join(word for word in text.split() if word not in stop_words)

df['text'] = df['text'].apply(remove_stop_words)
df['title'] = df['title'].apply(remove_stop_words)

[nltk_data] Downloading package stopwords to /home/rachel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
print(stop_words)

{'any', 'both', "it's", 'wouldn', 'but', 'very', 'him', "you'd", 'over', 'he', "you'll", 'myself', 'between', "don't", 'they', 'weren', 'down', 'hadn', "isn't", 'me', "doesn't", 'haven', "didn't", 'each', 'until', 'our', "wasn't", 'won', "needn't", 'ourselves', 'has', 'few', 'hers', 'am', 'up', 'all', "aren't", 'because', 'mustn', "shouldn't", 'those', 'yourselves', 'during', 'only', 'too', 'yours', 'why', 'can', 'o', 'my', 'such', 'about', 'his', "weren't", 'of', "mightn't", 'was', 'its', 'before', 'an', 'to', 'them', 'further', 'there', 'no', 'needn', "shan't", "hasn't", "should've", 'ma', "wouldn't", 'at', 'be', "you've", 'have', 'against', 'in', 'nor', 't', 'when', 'hasn', 'mightn', 'where', 'theirs', 'some', 'or', 'didn', 'm', 'who', 'out', 'most', 'don', "that'll", 'a', 'being', 'her', 'after', 'isn', 'll', 'again', 'aren', 'under', 'themselves', 'into', 'it', 'than', 'and', "haven't", 'having', 'which', 'do', 'ain', 'their', 'y', 'she', 'your', 'been', 'below', 'shan', "hadn't",

In [11]:
print(df)

                                                 title  \
0    law enforcement high alert following threats c...   
2    unbelievable obamas attorney general says char...   
3    bobby jindal raised hindu uses story christian...   
4    satan 2 russia unvelis image terrifying new su...   
5    time christian group sues amazon splc designat...   
..                                                 ...   
119         new york city budget boss state piggy bank   
120  long voting lines could longterm consequences ...   
121                  trump advisors try pivot fishtail   
122                    hillary land prison oval office   
123  zimbabwes mugabe resigns ending four decades rule   

                                                  text  label  
0    comment expected barack obama members fyf911 f...      1  
2    demonstrators gathered last night exercising c...      1  
3    dozen politically active pastors came private ...      0  
4    rs28 sarmat missile dubbed satan 2 replace

In [12]:
#tokenizing and lemmatizing the text with POS (took around 6 mins to run)

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag

lemmatizer = WordNetLemmatizer()

def wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return 'a'
    elif treebank_tag.startswith('V'):
        return 'v' 
    elif treebank_tag.startswith('N'):
        return 'n' 
    elif treebank_tag.startswith('R'):
        return 'r'  
    else:
        return 'n'  

def pos_lemmatize(text):
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(token, wordnet_pos(tag)) for token, tag in pos_tags]
    return ' '.join(lemmatized_tokens)

df['text'] = df['text'].apply(lambda x: pos_lemmatize(str(x)))
df['title'] = df['title'].apply(lambda x: pos_lemmatize(str(x)))

df.head()

,title,text,label
0,law enforcement high alert follow threat cop w...,comment expect barack obama member fyf911 fuky...,1
2,unbelievable obamas attorney general say charl...,demonstrator gather last night exercise consti...,1
3,bobby jindal raise hindu us story christian co...,dozen politically active pastor come private d...,0
4,satan 2 russia unvelis image terrify new super...,rs28 sarmat missile dub satan 2 replace ss18 f...,1
5,time christian group sue amazon splc designati...,say one time someone sue southern poverty law ...,1


In [14]:
#split the data up into train, val, and test

%pip install scikit-learn
import sklearn
from sklearn.model_selection import train_test_split

#combining the title with the text of article
df['combined'] = df['title'].fillna('') + ' ' + df['text'].fillna('')


df.to_csv('combined.csv', index=False)

df.head()

Note: you may need to restart the kernel to use updated packages.


,title,text,label,combined
0,law enforcement high alert follow threat cop w...,comment expect barack obama member fyf911 fuky...,1,law enforcement high alert follow threat cop w...
2,unbelievable obamas attorney general say charl...,demonstrator gather last night exercise consti...,1,unbelievable obamas attorney general say charl...
3,bobby jindal raise hindu us story christian co...,dozen politically active pastor come private d...,0,bobby jindal raise hindu us story christian co...
4,satan 2 russia unvelis image terrify new super...,rs28 sarmat missile dub satan 2 replace ss18 f...,1,satan 2 russia unvelis image terrify new super...
5,time christian group sue amazon splc designati...,say one time someone sue southern poverty law ...,1,time christian group sue amazon splc designati...


In [15]:
df = pd.read_csv('combined.csv')
df.head()

,title,text,label,combined
0,law enforcement high alert follow threat cop w...,comment expect barack obama member fyf911 fuky...,1,law enforcement high alert follow threat cop w...
1,unbelievable obamas attorney general say charl...,demonstrator gather last night exercise consti...,1,unbelievable obamas attorney general say charl...
2,bobby jindal raise hindu us story christian co...,dozen politically active pastor come private d...,0,bobby jindal raise hindu us story christian co...
3,satan 2 russia unvelis image terrify new super...,rs28 sarmat missile dub satan 2 replace ss18 f...,1,satan 2 russia unvelis image terrify new super...
4,time christian group sue amazon splc designati...,say one time someone sue southern poverty law ...,1,time christian group sue amazon splc designati...


In [ ]:
X = df['combined']  
y = df['label']  

#training data
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

#rest is validation and testing
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(X_train)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

#training data will create a vocabulary
vectorizer.fit(X_train)

training_text_vector = vectorizer.transform(X_train)
validation_text_vector = vectorizer.transform(X_val)
test_text_vector = vectorizer.transform(X_test)

In [ ]:

import torch
from scipy.sparse import csr_matrix

#need to use sparse matrix because the dense versions take too much memory

def matrix_to_sparse_tensor(matrix):
    coo = matrix.tocoo()
    indices = torch.tensor([coo.row, coo.col], dtype=torch.long)
    values = torch.tensor(coo.data, dtype=torch.float32)
    size = torch.Size(coo.shape)
    
    return torch.sparse_coo_tensor(indices, values, size)

train_feature_tensor = matrix_to_sparse_tensor(training_text_vector)
val_feature_tensor = matrix_to_sparse_tensor(validation_text_vector)
test_feature_tensor = matrix_to_sparse_tensor(test_text_vector)

In [ ]:
train_label_tensor = torch.tensor(y_train.values, dtype=torch.long)
val_label_tensor = torch.tensor(y_val.values, dtype=torch.long)
test_label_tensor = torch.tensor(y_test.values, dtype=torch.long)


In [ ]:
print(train_feature_tensor.shape)
print(val_feature_tensor.shape)
print(test_feature_tensor.shape)
print(train_feature_tensor)


In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
    
        if isinstance(feature, torch.sparse.Tensor):
            feature = feature.to_dense()
            
        return feature, label

In [ ]:
from torch.utils.data import DataLoader

train_custom_dataset = CustomDataset(train_feature_tensor, train_label_tensor)
val_custom_dataset = CustomDataset(val_feature_tensor, val_label_tensor)
test_custom_dataset = CustomDataset(test_feature_tensor, test_label_tensor)

#maybe try 32 or 64?
batch_size = 16

#create loaders for the train val and test data
train_loader = DataLoader(train_custom_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_custom_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_custom_dataset, batch_size=batch_size, shuffle=False)

In [ ]:

import torch.nn as nn

#define LSTM cell
class Cell(nn.Module):
    def __init__(self, input_dimension, hidden_dimension):
        super(Cell, self).__init__()
        self.hidden_dimension = hidden_dimension
        
        #we have the input, forget, and output gates. we also have the cell state
        self.W_input_gate = nn.Linear(input_dimension + hidden_dimension, hidden_dimension)
        self.W_forget_gate = nn.Linear(input_dimension + hidden_dimension, hidden_dimension)
        self.W_o_output_gate = nn.Linear(input_dimension + hidden_dimension, hidden_dimension)
        self.W_Cell_state_update = nn.Linear(input_dimension + hidden_dimension, hidden_dimension)
        
    def forward(self, x, hidden):
        
        prev_hidden, prev_cell_state = hidden

        #first the current cell and previous hidden cell are concatenated
        combined = torch.cat((x, prev_hidden), dim=1)
        
        #apply the transformations to the gates and cell state
        input = torch.sigmoid(self.W_input_gate(combined))
        forget = torch.sigmoid(self.W_forget_gate(combined))
        output = torch.sigmoid(self.W_o_output_gate(combined))
        cell_state = torch.tanh(self.W_Cell_state_update(combined))
        
        #calculate the new cell and hidden state
        new_cell = forget * prev_cell_state + input * cell_state
        new_hidden_state = output * torch.tanh(new_cell)
        
        return new_hidden_state, new_cell


In [ ]:

import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_dimension, hidden_dimension, output_dimension, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_dimension = hidden_dimension
        self.num_layers = num_layers  # Define num_layers here

        # Create a list of LSTM cells
        self.cells = nn.ModuleList(
            [nn.LSTM(input_dimension if i == 0 else hidden_dimension, hidden_dimension) for i in range(num_layers)]
        )
        self.fully_connected_layer = nn.Linear(hidden_dimension, output_dimension)

    def forward(self, x):
        batch_size = x.size(0)
        h = [torch.zeros(batch_size, self.hidden_dimension) for _ in range(self.num_layers)]
        c = [torch.zeros(batch_size, self.hidden_dimension) for _ in range(self.num_layers)]

        for t in range(x.size(1)):
            for i, cell in enumerate(self.cells):
                x, (h[i], c[i]) = cell(x, (h[i], c[i]))
        
        output = self.fully_connected_layer(x)
        return output

In [ ]:
for inputs, labels in train_loader:
    print(inputs, labels)

In [ ]:
import torch
import torch.optim as optim
from sklearn.metrics import accuracy_score

# Define model parameters
input_dimension = 291416  # Example feature dimension
hidden_dimension = 128
output_dimension = 2  # For binary classification
num_layers = 2

# Instantiate the model
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
model = LSTMModel(input_dimension, hidden_dimension, output_dimension, num_layers).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            # Move data to GPU
            inputs, labels = inputs.to(device), labels.to(device)
            print(inputs.shape)

            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(dataloader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

# Evaluation loop
def evaluate(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            # Move data to GPU
            inputs = inputs.to_dense()
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Accuracy: {accuracy:.4f}')

# Example usage:
# Assuming you have DataLoaders named `train_loader` and `test_loader`

# Train the model
train(model, train_loader, criterion, optimizer, num_epochs=10)

# Evaluate the model
evaluate(model, test_loader)